##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Addons Layers: Temporal Convolutional Network

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/temporal_convolutional_network"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/temporal_convolutional_network.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/addons/blob/master/docs/tutorials/temporal_convolutional_network.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/addons/docs/tutorials/temporal_convolutional_network.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview
This notebook will demonstrate how to use Temporal Convolutional Network in TensorFlow Addons.

## Setup

In [36]:
from __future__ import absolute_import, division, print_function

In [2]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

import tensorflow.keras as keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence

## Global Configurations

In [29]:
max_features = 20000
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

## Import IMDB data

In [30]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# pad the training data
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

## Build Simple TCN Model

In [31]:
# build the model using functional API
i = Input(shape=(maxlen,))
x = Embedding(max_features, 128)(i)
x = tfa.layers.TCN()(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[i], outputs=[x])

### Summary of the model

In [32]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
tcn_5 (TCN)                  (None, 64)                148800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 2,708,865
Trainable params: 2,708,865
Non-trainable params: 0
_________________________________________________________________


## Compute the model

In [33]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

## Fit the model

In [35]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 90s 4ms/sample - loss: 0.2968 - accuracy: 0.8742 - val_loss: 0.4443 - val_accuracy: 0.7968
Epoch 2/3
25000/25000 [==============================] - 90s 4ms/sample - loss: 0.1521 - accuracy: 0.9418 - val_loss: 0.5506 - val_accuracy: 0.8182
Epoch 3/3
25000/25000 [==============================] - 90s 4ms/sample - loss: 0.0779 - accuracy: 0.9721 - val_loss: 0.5611 - val_accuracy: 0.8075
